In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


In [2]:
data = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')

In [3]:
data.sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
97055,074cbd6330d4dd8a,There is a vertical template on Wikipedia talk...,0,0,0,0,0,0
29118,4d2cfd4c0a7a7452,GLAM Cafe 11-August-2015 \nN.B. that the GLAM ...,0,0,0,0,0,0
95562,ff9114bdf09b789f,Good point. Except that content is based on ci...,0,0,0,0,0,0
147552,3e1c663acfead4df,You repeat maasive removal on Nguyễn Dynasty a...,0,0,0,0,0,0
6845,12444e47cd055268,LOL. You are a moron.,1,0,1,0,1,0
40556,6c3c769e8c776c89,Hey you deleted the pics i put up for the mali...,1,0,1,0,0,0
74644,c7b59a3ec4f1595e,"Well before we consider including your image, ...",0,0,0,0,0,0
57691,9a614829d29a06ca,Articles for Deletion \n\nI realize you are no...,0,0,0,0,0,0
42696,71f0af4d1ba5ce09,"""\n\n Thank you for confirming that you cannot...",0,0,0,0,0,0
59064,9e327c92bf0383ca,"As I see it, you aren't reading my sources. I...",0,0,0,0,0,0


In [4]:
cols = data.columns[2:]

In [5]:
toxic_comments = data[data[cols].sum(axis=1)!=0]
safe_comments = data[data[cols].sum(axis=1)==0]

In [6]:
toxic_comments = toxic_comments.sample(frac=1)
safe_comments = safe_comments.sample(frac=1)

In [7]:
len(toxic_comments), len(safe_comments)

(16225, 143346)

In [8]:
safe_split = 0.3
toxic_split = 0.2
train_safe = safe_comments.iloc[int(len(safe_comments)*safe_split):]
test_safe = safe_comments.iloc[:int(len(safe_comments)*safe_split)]

train_toxic = toxic_comments.iloc[int(len(toxic_comments)*toxic_split):]
test_toxic = toxic_comments.iloc[:int(len(toxic_comments)*toxic_split)]

In [9]:
val_safe = test_safe.iloc[int(len(test_safe)*0.5):]
test_safe = test_safe.iloc[:int(len(test_safe)*0.5)]

val_toxic = test_toxic.iloc[int(len(test_toxic)*0.5):]
test_toxic = test_toxic.iloc[:int(len(test_toxic)*0.5)]

In [10]:
train_df = train_safe.append(train_toxic)
val_df = val_safe.append(val_toxic)
test_df = test_safe.append(test_toxic)

In [11]:
def balance(df, smaller_df,imb_ratio):
    for i in range(imb_ratio-1):
        df = df.append(smaller_df)
        df = df.sample(frac=1)
    return df.sample(frac=1)

In [12]:
train_toxic[cols].sum()

toxic            12227
severe_toxic      1287
obscene           6746
threat             365
insult            6299
identity_hate     1116
dtype: int64

In [13]:
threats = train_df.query("threat == 1")
id_hate = train_df.query("identity_hate == 1")
sev_toxic = train_df.query("severe_toxic == 1")
insult = train_df.query("insult == 1")
obsc = train_df.query("obscene == 1")
tox = train_df.query("toxic == 1")

In [14]:
new_bal = balance(train_df, threats, 25)
new_bal = balance(new_bal, id_hate, 20)
new_bal = balance(new_bal, sev_toxic, 20)
new_bal = balance(new_bal, insult, 2)
new_bal = balance(new_bal, obsc, 2)
new_bal = balance(new_bal, tox, 2)
new_bal[cols].sum(), new_bal.shape

(toxic            88857
 severe_toxic     36079
 obscene          69118
 threat           13025
 insult           67476
 identity_hate    31670
 dtype: int64,
 (193012, 8))

In [15]:
# train_bal = balance(train_df, train_toxic, 10)
train_bal = new_bal
val_bal = balance(val_df, val_toxic, 5)
test_bal = balance(test_df, test_toxic, 5)

In [16]:
train_bal[train_bal[cols].sum(axis=1)!=0].shape,\
train_bal[train_bal[cols].sum(axis=1)==0].shape

((92669, 8), (100343, 8))

In [17]:
val_bal[cols].sample(10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
62870,0,0,0,0,0,0
38252,0,0,0,0,0,0
93393,0,0,0,0,0,0
7226,0,0,0,0,0,0
103300,0,0,0,0,0,0
89886,0,0,0,0,0,0
17843,1,0,1,0,1,0
47990,1,0,0,0,0,0
96567,0,0,0,0,0,0
102787,0,0,0,0,0,0


In [18]:
X_train = train_bal['comment_text'].values
y_train = train_bal[cols].values
X_val = val_bal['comment_text'].values
y_val = val_bal[cols].values
X_test = test_bal['comment_text'].values
y_test = test_bal[cols].values

In [19]:
X_train.shape, X_val.shape, X_test.shape

((193012,), (29617,), (29611,))

In [21]:
from tensorflow.keras.layers import LSTM, Input, Dense, TextVectorization, Dropout, Bidirectional, Embedding

In [25]:
VOCAB_SIZE = 200000
tx = 1800
vectorizer = TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=tx)

In [29]:
vectorizer.adapt(X_train)

TypeError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/base_preprocessing_layer.py", line 123, in adapt_step  *
        self.update_state(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/layers/preprocessing/text_vectorization.py", line 475, in update_state  **
        self._lookup_layer.update_state(self._preprocess(data))
    File "/opt/conda/lib/python3.7/site-packages/keras/layers/preprocessing/text_vectorization.py", line 552, in _preprocess
        inputs = utils.ensure_tensor(inputs, dtype=tf.string)
    File "/opt/conda/lib/python3.7/site-packages/keras/layers/preprocessing/preprocessing_utils.py", line 32, in ensure_tensor
        inputs = tf.convert_to_tensor(inputs, dtype)

    TypeError: Cannot convert a list containing a tensor of dtype <dtype: 'int64'> to <dtype: 'string'> (Tensor is: <tf.Tensor 'IteratorGetNext:1' shape=(None, 6) dtype=int64>)


In [23]:
idx=45
vectorizer(X_val[idx]), X_val[idx]

(<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([15, 40, 31, ...,  0,  0,  0])>,
 '"\nThis page will show events that occurred worldwide in rail transport during the month, just like pages for previous months.  Most of the updates to this page are made when Portal:Trains is updated to include new items in the ""In the news"" section.  The section itself hasn\'t been updated yet this month, but there are plenty of items to add to this page as evidenced on the associated ""In the news"" talk page.   (Speak) "')

In [24]:
train_vect = vectorizer(X_train)
val_vect = vectorizer(X_val)
test_vect = vectorizer(X_test)

In [27]:
from tensorflow.data import Dataset
import tensorflow as tf

train_dataset = Dataset.from_tensor_slices((X_train, y_train))
val_dataset = Dataset.from_tensor_slices((X_val, y_val))
test_dataset = Dataset.from_tensor_slices((X_test, y_test))

In [22]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache()
val_dataset = val_dataset.cache()
test_dataset = test_dataset.cache()
train_dataset = train_dataset.shuffle(200000)  
val_dataset = val_dataset.shuffle(30000)
test_dataset = test_dataset.shuffle(30000)
train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)
test_dataset = test_dataset.batch(32)
train_dataset = train_dataset.prefetch(AUTOTUNE)
val_dataset = val_dataset.prefetch(AUTOTUNE)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [23]:
len(train_dataset), len(val_dataset), len(test_dataset)

(6032, 926, 926)

In [28]:
inputs = Input(tx)
x = Embedding(VOCAB_SIZE+1, 32)(inputs)
x = Bidirectional(LSTM(64, activation='tanh'))(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(6, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

In [29]:
# optim = tf.keras.optimizers.Adam(0.01)
model.compile(loss='binary_crossentropy', optimizer='adam')

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1800)]            0         
                                                                 
 embedding (Embedding)       (None, 1800, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                             

In [31]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('loss')<0.03:
            print('Threshold reached')
            self.model.stop_training = True

callback = mycallback()

In [32]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=2, callbacks=[callback])

Epoch 1/2
6022/6022 [==============================] - 1268s 209ms/step - loss: 0.1668 - val_loss: 0.1500
Epoch 2/2
6022/6022 [==============================] - 1161s 193ms/step - loss: 0.0657 - val_loss: 0.2073


In [34]:
import pickle
model.save('bi_lstm_final.h5')
pickle.dump({'config': vectorizer.get_config(),
             'weights': vectorizer.get_weights()}
            , open("vec_layer.pkl", "wb"))

In [ ]:
data.columns[2:]

In [37]:
prompt = "lovely man"
prompt_vec= vectorizer(prompt)
res = model.predict(np.expand_dims(prompt_vec,0))
print(data.columns[2:])
print(res>0.5)

1/1 [==============================] - 0s 66ms/step
Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')
[[False False False False False False]]


In [38]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

precision = Precision()
recall = Recall()
accuracy = CategoricalAccuracy()

In [39]:
for batch in test_dataset.as_numpy_iterator():
    X_true, y_true = batch
    y_pred = model.predict(X_true)
    
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    precision.update_state(y_true, y_pred)
    recall.update_state(y_true, y_pred)
    accuracy.update_state(y_true, y_pred)

1/1 [==============================] - 0s 56ms/step


In [40]:
f'precision: {precision.result().numpy()}, \
recall:{recall.result().numpy()}, \
accuracy: {accuracy.result().numpy()}'

'precision: 0.7972587943077087, recall:0.6606690883636475, accuracy: 0.12634989619255066'

In [43]:
# model = tf.keras.models.load_model('/kaggle/working/bi_lstm_final.h5', compile=False)
# from_disk = pickle.load(open("/kaggle/working/vec_layer.pkl", "rb"))
# vectorizer = tf.keras.layers.TextVectorization.from_config(from_disk['config'])
# vectoriz.set_weights(from_disk['weights'])